In [2]:
import numpy as np 
import pandas as pd

In [4]:
def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))

    # set beta(t) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
    # beta is a backwaard param: answers the prob we are going to be
    # at the state si knowing what is going to come in the future and not
    # in the past. 
    # loop back from t-2 to 0
    for t in range(V.shape[0]-2,-1,-1):
        for j in range(a.shape[0]):
            beta[t,j] = (beta[t+1]*b[:,V[t+1]]).dot(a[j, :])
    return beta 

def forward(V, a, b, initial_distribution):
    # alpha is a forward param: answers what is the probability that
    # we are in this particular state given the set of observations we
    # have seen so far
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]

    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            alpha[t,j] = alpha[t-1].dot(a[:,j])*b[j, V[t]]
    return alpha

In [5]:
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)
 
    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
        # gamma is a fusion of alpha & beta :  

        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return {"a":a, "b":b}

In [7]:
data = pd.read_csv('./data_python.csv')
V = data['Visible'].values

# transition probabilitiew
a = np.ones((2,2))
a = a/np.sum(a, axis=1)

# emission probabilities
b = np.array((
    (1,3,5),
    (2,4,6)))
b = b/np.sum(b, axis=1).reshape(-1,1)

# equal probabilities for initial distribution
initial_distribution = np.array((0.5,0.5))

print(baum_welch(V,a,b,initial_distribution, n_iter=100))

{'a': array([[0.53816345, 0.46183655],
       [0.48664443, 0.51335557]]), 'b': array([[0.16277513, 0.26258073, 0.57464414],
       [0.2514996 , 0.27780971, 0.47069069]])}
